MORTAGE BACKED SECURITIES [MBS]:

A Mortage Backed securities [MBS] is An investment similar to a bond that is made up of bundle of home loans bought from the banks that issued them. in this system, the loans issued by the bank is in turn sold to investors at a discounted rate to free up the bank funds. these loans are sold in the form of bonds by investment banks wherein loans are grouped togetter according to their type type and quality. for the investor, an MBS is safe as the mortage loans that back it up

PREPAYMENT RISK:

prepayment is the risk involed with the premature return of the principle on a fixed income security. when debtors return part of the principle early, they do not have to make intreset payments on that part of the principle.
that means the loan issuer prepays the loan, the investors will stop receiving the intreset on these bonds. hence, it is important to evalute the prepayment risks on the MBSand thus this is the aim of the project.

LIBRARIES:

In [2]:
import os
import numpy as np
import pandas as pd
import warnings

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [5]:
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score as R2
from sklearn.model_selection  import cross_val_score as CVS

In [6]:
from sklearn.linear_model import LinearRegression

In [7]:
sns.set(context='notebook', style='darkgrid', palette='deep', font='sans-serif', font_scale=1, color_codes=False, rc=None)
pd.options.display.max_columns

20

In [8]:
warnings.filterwarnings("always")
warnings.filterwarnings("ignore")

FILE PATHS:

In [9]:

for dirname, _, filenames in os.walk('/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [10]:

tr_path = "/content/LoanExport.csv"

DATA PRE-PROCESSING:
in the pre-processing step the data is converted into consumable format for the machine learning models. The X and Y models are defined, labelling and encoding is performed if required and the dataset is spilt into the training and cross validation and test set to gauge the varied performance of the machine learning models and to evalute the performance accordingly.


In [16]:
# read in csv file as a DataFrame
tr_df = pd.read_csv("/content/LoanExport.csv")
# explore the first 5 rows
tr_df.head()

,CreditScore,FirstPaymentDate,FirstTimeHomebuyer,MaturityDate,MSA,MIP,Units,Occupancy,OCLTV,DTI,...,PostalCode,LoanSeqNum,LoanPurpose,OrigLoanTerm,NumBorrowers,SellerName,ServicerName,EverDelinquent,MonthsDelinquent,MonthsInRepayment
0,0,199902,N,202901,16974,25,1,O,89,27,...,60400,F199Q1268030,P,360,2,FL,WASHINGTONMUTUALBANK,0.0,0.0,52.0
1,0,199902,N,202901,19740,0,1,O,73,17,...,80200,F199Q1015092,N,360,1,FT,CHASEHOMEFINANCELLC,0.0,0.0,144.0
2,0,199902,N,202901,29940,0,1,O,75,16,...,66000,F199Q1266886,N,360,2,FL,WASHINGTONMUTUALBANK,0.0,0.0,67.0
3,0,199902,N,202901,31084,0,1,O,76,14,...,90700,F199Q1178167,N,360,2,GM,GMACMTGECORP,0.0,0.0,35.0
4,0,199902,N,202901,35644,0,1,O,78,18,...,7600,F199Q1178517,N,360,2,GM,GMACMTGECORP,0.0,0.0,54.0


In [17]:
#column information
tr_df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8288 entries, 0 to 8287
Data columns (total 28 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CreditScore         8288 non-null   int64  
 1   FirstPaymentDate    8288 non-null   int64  
 2   FirstTimeHomebuyer  8288 non-null   object 
 3   MaturityDate        8288 non-null   int64  
 4   MSA                 8288 non-null   object 
 5   MIP                 8288 non-null   int64  
 6   Units               8288 non-null   int64  
 7   Occupancy           8288 non-null   object 
 8   OCLTV               8288 non-null   int64  
 9   DTI                 8288 non-null   int64  
 10  OrigUPB             8288 non-null   int64  
 11  LTV                 8288 non-null   int64  
 12  OrigInterestRate    8288 non-null   float64
 13  Channel             8288 non-null   object 
 14  PPM                 8288 non-null   object 
 15  ProductType         8288 non-null   object 
 16  Proper

In [18]:
tr_df.describe()

,CreditScore,FirstPaymentDate,MaturityDate,MIP,Units,OCLTV,DTI,OrigUPB,LTV,OrigInterestRate,PostalCode,OrigLoanTerm,EverDelinquent,MonthsDelinquent,MonthsInRepayment
count,8288.000000,8288.000000,8288.000000,8288.000000,8288.000000,8288.000000,8288.000000,8288.000000,8288.000000,8288.000000,8288.000000,8288.000000,8287.000000,8287.000000,8287.000000
mean,497.931226,199904.920125,202901.307915,12.749035,1.029199,80.860763,29.553571,121071.790541,80.845922,7.053185,53255.091699,359.702461,0.440087,6.433088,63.205623
std,205.132493,12.703044,24.739000,13.259903,0.208117,13.005811,13.156933,52854.719458,13.010259,0.440372,30311.163141,3.006374,0.496427,16.473469,50.595431
min,0.000000,199902.000000,202409.000000,0.000000,1.000000,7.000000,0.000000,13000.000000,7.000000,4.875000,600.000000,306.000000,0.000000,0.000000,1.000000
25%,552.000000,199903.000000,202902.000000,0.000000,1.000000,75.000000,22.000000,80000.000000,75.000000,6.875000,29500.000000,360.000000,0.000000,0.000000,33.000000
50%,584.000000,199904.000000,202903.000000,12.000000,1.000000,80.000000,31.000000,115000.000000,80.000000,7.000000,48300.000000,360.000000,0.000000,0.000000,49.000000
75%,599.000000,199905.000000,202904.000000,25.000000,1.000000,90.000000,39.000000,154250.000000,90.000000,7.250000,84100.000000,360.000000,1.000000,4.000000,69.000000
max,607.000000,200304.000000,203010.000000,52.000000,4.000000,103.000000,65.000000,400000.000000,100.000000,9.750000,99700.000000,360.000000,1.000000,180.000000,212.000000


DATA CLEANING: in this step the dta is primarily cleaned first, this means the rows with null values are removed, the unwanted columns are dropped from the dataset etc. so that we are able to proscess the data better.

In [19]:
#missing values in decsending order
print(tr_df.isnull().sum().sort_values(ascending=False),"\n\n",tr_df.isnull().sum()/tr_df.shape[0] *100,"\n\n")


SellerName            782
MonthsInRepayment       1
MonthsDelinquent        1
EverDelinquent          1
ServicerName            1
FirstPaymentDate        0
NumBorrowers            0
OrigLoanTerm            0
LoanPurpose             0
LoanSeqNum              0
PostalCode              0
PropertyType            0
PropertyState           0
ProductType             0
CreditScore             0
Channel                 0
OrigInterestRate        0
LTV                     0
OrigUPB                 0
DTI                     0
OCLTV                   0
Occupancy               0
Units                   0
MIP                     0
MSA                     0
MaturityDate            0
FirstTimeHomebuyer      0
PPM                     0
dtype: int64 

 CreditScore           0.000000
FirstPaymentDate      0.000000
FirstTimeHomebuyer    0.000000
MaturityDate          0.000000
MSA                   0.000000
MIP                   0.000000
Units                 0.000000
Occupancy             0.000000
OCLTV   

In [24]:
tr_df.dropna(subset=['SellerName'], inplace=True)

In [25]:
tr_df.head()

,CreditScore,FirstPaymentDate,FirstTimeHomebuyer,MaturityDate,MSA,MIP,Units,Occupancy,OCLTV,DTI,...,PostalCode,LoanSeqNum,LoanPurpose,OrigLoanTerm,NumBorrowers,SellerName,ServicerName,EverDelinquent,MonthsDelinquent,MonthsInRepayment
0,0,199902,N,202901,16974,25,1,O,89,27,...,60400,F199Q1268030,P,360,2,FL,WASHINGTONMUTUALBANK,0.0,0.0,52.0
1,0,199902,N,202901,19740,0,1,O,73,17,...,80200,F199Q1015092,N,360,1,FT,CHASEHOMEFINANCELLC,0.0,0.0,144.0
2,0,199902,N,202901,29940,0,1,O,75,16,...,66000,F199Q1266886,N,360,2,FL,WASHINGTONMUTUALBANK,0.0,0.0,67.0
3,0,199902,N,202901,31084,0,1,O,76,14,...,90700,F199Q1178167,N,360,2,GM,GMACMTGECORP,0.0,0.0,35.0
4,0,199902,N,202901,35644,0,1,O,78,18,...,7600,F199Q1178517,N,360,2,GM,GMACMTGECORP,0.0,0.0,54.0


In [27]:
#lable encoding

le = LabelEncoder()
Label = ['MonthsInRepayment','PostalCode','SellerName']

for i in Label:
    tr_df[i] = le.fit_transform(tr_df[i])

In [28]:
tr_df.head()

,CreditScore,FirstPaymentDate,FirstTimeHomebuyer,MaturityDate,MSA,MIP,Units,Occupancy,OCLTV,DTI,...,PostalCode,LoanSeqNum,LoanPurpose,OrigLoanTerm,NumBorrowers,SellerName,ServicerName,EverDelinquent,MonthsDelinquent,MonthsInRepayment
0,0,199902,N,202901,16974,25,1,O,89,27,...,461,F199Q1268030,P,360,2,7,WASHINGTONMUTUALBANK,0.0,0.0,51
1,0,199902,N,202901,19740,0,1,O,73,17,...,590,F199Q1015092,N,360,1,8,CHASEHOMEFINANCELLC,0.0,0.0,143
2,0,199902,N,202901,29940,0,1,O,75,16,...,502,F199Q1266886,N,360,2,7,WASHINGTONMUTUALBANK,0.0,0.0,66
3,0,199902,N,202901,31084,0,1,O,76,14,...,655,F199Q1178167,N,360,2,10,GMACMTGECORP,0.0,0.0,34
4,0,199902,N,202901,35644,0,1,O,78,18,...,60,F199Q1178517,N,360,2,10,GMACMTGECORP,0.0,0.0,53


In [30]:
#one hot encoding
cols = ['ServicerName','LoanSeqNum']
# Apply one-hot encoder
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
tr_oh = pd.DataFrame(OH_encoder.fit_transform(tr_df[cols])).astype('int64')


In [32]:
tr_df.head()

,CreditScore,FirstPaymentDate,FirstTimeHomebuyer,MaturityDate,MSA,MIP,Units,Occupancy,OCLTV,DTI,...,PostalCode,LoanSeqNum,LoanPurpose,OrigLoanTerm,NumBorrowers,SellerName,ServicerName,EverDelinquent,MonthsDelinquent,MonthsInRepayment
0,0,199902,N,202901,16974,25,1,O,89,27,...,461,F199Q1268030,P,360,2,7,WASHINGTONMUTUALBANK,0.0,0.0,51
1,0,199902,N,202901,19740,0,1,O,73,17,...,590,F199Q1015092,N,360,1,8,CHASEHOMEFINANCELLC,0.0,0.0,143
2,0,199902,N,202901,29940,0,1,O,75,16,...,502,F199Q1266886,N,360,2,7,WASHINGTONMUTUALBANK,0.0,0.0,66
3,0,199902,N,202901,31084,0,1,O,76,14,...,655,F199Q1178167,N,360,2,10,GMACMTGECORP,0.0,0.0,34
4,0,199902,N,202901,35644,0,1,O,78,18,...,60,F199Q1178517,N,360,2,10,GMACMTGECORP,0.0,0.0,53


DATA LABELLING:

In [33]:
import pandas as pd

def label_data(row):
    # Write your labeling logic here
    loan_amount = row['LoanAmount']
    if loan_amount > 50000:
        label = 'High'
    elif loan_amount > 20000:
        label = 'Medium'
    else:
        label = 'Low'

    return label

In [34]:
tr_df.head()

,CreditScore,FirstPaymentDate,FirstTimeHomebuyer,MaturityDate,MSA,MIP,Units,Occupancy,OCLTV,DTI,...,PostalCode,LoanSeqNum,LoanPurpose,OrigLoanTerm,NumBorrowers,SellerName,ServicerName,EverDelinquent,MonthsDelinquent,MonthsInRepayment
0,0,199902,N,202901,16974,25,1,O,89,27,...,461,F199Q1268030,P,360,2,7,WASHINGTONMUTUALBANK,0.0,0.0,51
1,0,199902,N,202901,19740,0,1,O,73,17,...,590,F199Q1015092,N,360,1,8,CHASEHOMEFINANCELLC,0.0,0.0,143
2,0,199902,N,202901,29940,0,1,O,75,16,...,502,F199Q1266886,N,360,2,7,WASHINGTONMUTUALBANK,0.0,0.0,66
3,0,199902,N,202901,31084,0,1,O,76,14,...,655,F199Q1178167,N,360,2,10,GMACMTGECORP,0.0,0.0,34
4,0,199902,N,202901,35644,0,1,O,78,18,...,60,F199Q1178517,N,360,2,10,GMACMTGECORP,0.0,0.0,53


In [37]:
def label_loan_data(csv_file):
    # Load the CSV file into a pandas DataFrame
    tr_df = pd.read_csv("/content/LoanExport.csv")

In [46]:
def label_loan_data(csv_file):
    # Load the CSV file into a pandas DataFrame
    tr_df = pd.read_csv(csv_file)

    # Add a new column 'Label' to store the labels
    tr_df['Label'] = tr_df.apply(label_data, axis=1)

In [49]:
tr_df.head()

,CreditScore,FirstPaymentDate,FirstTimeHomebuyer,MaturityDate,MSA,MIP,Units,Occupancy,OCLTV,DTI,...,PostalCode,LoanSeqNum,LoanPurpose,OrigLoanTerm,NumBorrowers,SellerName,ServicerName,EverDelinquent,MonthsDelinquent,MonthsInRepayment
0,0,199902,N,202901,16974,25,1,O,89,27,...,461,F199Q1268030,P,360,2,7,WASHINGTONMUTUALBANK,0.0,0.0,51
1,0,199902,N,202901,19740,0,1,O,73,17,...,590,F199Q1015092,N,360,1,8,CHASEHOMEFINANCELLC,0.0,0.0,143
2,0,199902,N,202901,29940,0,1,O,75,16,...,502,F199Q1266886,N,360,2,7,WASHINGTONMUTUALBANK,0.0,0.0,66
3,0,199902,N,202901,31084,0,1,O,76,14,...,655,F199Q1178167,N,360,2,10,GMACMTGECORP,0.0,0.0,34
4,0,199902,N,202901,35644,0,1,O,78,18,...,60,F199Q1178517,N,360,2,10,GMACMTGECORP,0.0,0.0,53
